# Phase 3: Alpha Sources Validation

This notebook validates the Phase 3 alpha sources integration:
1. Reddit data fetching (public JSON endpoints)
2. FinBERT sentiment analysis with CARVS scoring
3. On-chain signal interpretation
4. Market regime classification
5. Combined alpha signal generation

**Pass Criteria:**
- Reddit fetching returns valid posts
- Sentiment analysis produces scores in [-1, 1]
- Regime classifier correctly classifies bull/bear scenarios
- Combined signals produce actionable trading recommendations

In [ ]:
# Setup
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import asyncio
from datetime import datetime

# Set style
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print(f"Project root: {project_root}")

## 1. Reddit Data Fetching

Testing the public JSON endpoint abstraction for Reddit data.

In [ ]:
from data.ingestion.reddit_sources import get_reddit_source, RedditPublicJSON

# Initialize Reddit source (uses public JSON by default)
reddit = get_reddit_source({})
print(f"Reddit source type: {type(reddit).__name__}")

# Fetch posts from Bitcoin subreddit
print("\nFetching posts from r/Bitcoin...")
posts = await reddit.fetch_subreddit_posts('Bitcoin', limit=10)

print(f"\nFetched {len(posts)} posts")
if posts:
    print("\nSample post structure:")
    sample = posts[0]
    for key, value in sample.items():
        if key == 'body':
            print(f"  {key}: {str(value)[:100]}..." if len(str(value)) > 100 else f"  {key}: {value}")
        elif key == 'title':
            print(f"  {key}: {str(value)[:80]}..." if len(str(value)) > 80 else f"  {key}: {value}")
        else:
            print(f"  {key}: {value}")

In [ ]:
# Fetch from multiple crypto subreddits
print("Fetching from multiple subreddits...")
multi_posts = await reddit.fetch_multiple_subreddits(limit_per_sub=5)

print(f"\nTotal posts: {len(multi_posts)}")

# Analyze post distribution
subreddit_counts = {}
for post in multi_posts:
    sub = post.get('subreddit', 'unknown')
    subreddit_counts[sub] = subreddit_counts.get(sub, 0) + 1

print("\nPosts by subreddit:")
for sub, count in sorted(subreddit_counts.items(), key=lambda x: -x[1]):
    print(f"  r/{sub}: {count} posts")

In [ ]:
# Display post titles for review
print("\nRecent Post Titles:")
print("="*70)
for i, post in enumerate(multi_posts[:15]):
    title = post['title'][:65] + '...' if len(post['title']) > 65 else post['title']
    score = post.get('score', 0)
    print(f"{i+1:2}. [{score:4}] {title}")

## 2. FinBERT Sentiment Analysis

Testing sentiment analysis with CARVS scoring.

In [ ]:
from data.ingestion.sentiment import FinBERTAnalyzer, SentimentAggregator

# Initialize FinBERT analyzer
print("Initializing FinBERT analyzer...")
analyzer = FinBERTAnalyzer()

# Test on sample texts
test_texts = [
    "Bitcoin is going to moon! Huge bullish breakout incoming!",
    "Market crash imminent. Sell everything now.",
    "BTC holding steady at support levels.",
    "Institutional adoption accelerating. Very bullish for crypto.",
    "Another exchange hack. This is why I don't trust crypto.",
]

print("\nSample Sentiment Analysis:")
print("="*70)
for text in test_texts:
    result = analyzer.analyze(text)
    # Calculate score as positive - negative for -1 to 1 scale
    score = result.positive_score - result.negative_score
    sentiment_label = 'Bullish' if score > 0.1 else 'Bearish' if score < -0.1 else 'Neutral'
    print(f"\n{text[:60]}..." if len(text) > 60 else f"\n{text}")
    print(f"  Score: {score:+.3f} | Confidence: {result.confidence:.3f} | {sentiment_label}")

In [ ]:
# Aggregate sentiment from Reddit posts
if multi_posts:
    print(f"\nAggregating sentiment from {len(multi_posts)} posts...")
    aggregator = SentimentAggregator(analyzer)
    
    aggregated = aggregator.aggregate_sentiment(multi_posts)
    
    print(f"\nAggregated Sentiment Results:")
    print(f"  Average score: {aggregated.avg_sentiment_score:+.4f}")
    print(f"  Volume-weighted score: {aggregated.volume_weighted_sentiment:+.4f}")
    print(f"  Sentiment std: {aggregated.sentiment_std:.4f}")
    print(f"  Bullish ratio: {aggregated.bullish_ratio:.2%}")
    print(f"  CARVS score: {aggregated.carvs_score:+.4f}")
    print(f"  Post count: {aggregated.num_posts}")
else:
    print("No posts available for aggregation")

In [ ]:
# Generate trading signal from sentiment
if multi_posts:
    signal = aggregator.generate_signal(aggregated)
    
    print("\nSentiment Trading Signal:")
    print(f"  Signal: {signal['signal']:+.4f}")
    print(f"  Confidence: {signal['confidence']:.4f}")
    print(f"  CARVS Score: {signal.get('carvs_score', 0):+.4f}")
    print(f"  Direction: {signal['direction']}")
    
    # Validate signal range
    assert -1 <= signal['signal'] <= 1, "Signal out of range!"
    assert 0 <= signal['confidence'] <= 1, "Confidence out of range!"
    print("\n  Signal values in valid range")

## 3. On-Chain Signal Interpretation

Testing on-chain metric interpretation (MVRV, SOPR, Netflows).

In [ ]:
from data.ingestion.onchain import OnChainSignalGenerator, ONCHAIN_METRICS
from unittest.mock import Mock

# Display supported metrics
print("Supported On-Chain Metrics:")
print("="*60)
for name, metric in ONCHAIN_METRICS.items():
    print(f"\n{name.upper()}")
    print(f"  {metric.description}")
    if metric.interpretation:
        print(f"  Interpretation: {metric.interpretation[:60]}...")

In [ ]:
# Create signal generator with mock provider
mock_provider = Mock()
generator = OnChainSignalGenerator(mock_provider)

# Test MVRV interpretation
print("\nMVRV Signal Interpretation:")
print("="*60)
mvrv_values = [8.0, 4.0, 2.5, 1.5, 0.0, -0.3, -1.0]

for mvrv in mvrv_values:
    result = generator.interpret_mvrv(mvrv)
    print(f"  MVRV {mvrv:+5.1f} -> Signal: {result['signal']:+.2f} | Regime: {result['regime']}")

In [ ]:
# Test SOPR interpretation
print("\nSOPR Signal Interpretation:")
print("="*60)
sopr_values = [1.15, 1.08, 1.03, 1.00, 0.97, 0.92]

for sopr in sopr_values:
    result = generator.interpret_sopr(sopr)
    print(f"  SOPR {sopr:.2f} -> Signal: {result['signal']:+.2f} | Regime: {result['regime']}")

In [ ]:
# Test Exchange Netflow interpretation
print("\nExchange Netflow Signal Interpretation:")
print("="*60)
netflow_values = [50000, 15000, 5000, 0, -5000, -15000, -50000]

for netflow in netflow_values:
    result = generator.interpret_exchange_netflow(netflow)
    print(f"  Netflow {netflow:+7} BTC -> Signal: {result['signal']:+.2f} | Regime: {result['regime']}")

In [ ]:
# Test combined on-chain signals
print("\nCombined On-Chain Signal (Bullish Scenario):")
print("="*60)

bullish_signals = {
    'mvrv': generator.interpret_mvrv(-0.3),  # Undervalued
    'sopr': generator.interpret_sopr(0.95),   # Capitulation
    'exchange_netflow': generator.interpret_exchange_netflow(-10000),  # Outflows
}

combined_bullish = generator.combine_signals(bullish_signals)
print(f"  Combined signal: {combined_bullish['signal']:+.3f}")
print(f"  Regime: {combined_bullish['regime']}")

print("\nCombined On-Chain Signal (Bearish Scenario):")
print("="*60)

bearish_signals = {
    'mvrv': generator.interpret_mvrv(5.0),   # Overvalued
    'sopr': generator.interpret_sopr(1.10),  # Profit taking
    'exchange_netflow': generator.interpret_exchange_netflow(20000),  # Inflows
}

combined_bearish = generator.combine_signals(bearish_signals)
print(f"  Combined signal: {combined_bearish['signal']:+.3f}")
print(f"  Regime: {combined_bearish['regime']}")

## 4. Market Regime Classification

Testing the regime classifier with various signal combinations.

In [ ]:
from models.predictors.regime_classifier import RegimeClassifier, MarketRegime, RegimeHistory

# Initialize classifier
classifier = RegimeClassifier()

print("Regime Classifier Configuration:")
print(f"  Weights: {classifier.weights}")
print(f"  Thresholds: {classifier.thresholds}")

In [ ]:
# Test classification with various scenarios
scenarios = [
    {'name': 'Strong Bull', 'onchain': 0.8, 'sentiment': 0.7, 'technical': 0.6},
    {'name': 'Bull', 'onchain': 0.4, 'sentiment': 0.3, 'technical': 0.4},
    {'name': 'Accumulation', 'onchain': 0.3, 'sentiment': 0.1, 'technical': 0.1},
    {'name': 'Neutral', 'onchain': 0.0, 'sentiment': 0.1, 'technical': -0.1},
    {'name': 'Distribution', 'onchain': -0.2, 'sentiment': -0.15, 'technical': -0.1},
    {'name': 'Bear', 'onchain': -0.4, 'sentiment': -0.3, 'technical': -0.4},
    {'name': 'Strong Bear', 'onchain': -0.8, 'sentiment': -0.7, 'technical': -0.6},
    {'name': 'Mixed Signals', 'onchain': 0.5, 'sentiment': -0.3, 'technical': 0.1},
]

print("\nRegime Classification Results:")
print("="*80)
print(f"{'Scenario':<15} {'Onchain':>8} {'Sent':>8} {'Tech':>8} -> {'Regime':<15} {'Conf':>6}")
print("-"*80)

for s in scenarios:
    result = classifier.classify(
        onchain_signal=s['onchain'],
        sentiment_signal=s['sentiment'],
        technical_signal=s['technical']
    )
    print(f"{s['name']:<15} {s['onchain']:+.2f}     {s['sentiment']:+.2f}    {s['technical']:+.2f}     -> "
          f"{result.regime.value:<15} {result.confidence:.2f}")

In [ ]:
# Test trading bias for each regime
print("\nTrading Bias by Regime:")
print("="*70)
print(f"{'Regime':<15} {'Bias':<8} {'Size Mult':>10} {'SL Mult':>10} {'TP Mult':>10}")
print("-"*70)

for regime in MarketRegime:
    bias = classifier.get_trading_bias(regime)
    print(f"{regime.value:<15} {bias['position_bias']:<8} "
          f"{bias['position_size_mult']:>10.1f} "
          f"{bias['stop_loss_mult']:>10.1f} "
          f"{bias['take_profit_mult']:>10.1f}")

In [ ]:
# Test technical signal calculation
print("\nTechnical Signal from Price Data:")
print("="*60)

# Generate sample trending price data
np.random.seed(42)
n = 100

# Uptrending prices
up_prices = pd.Series(100 * np.cumprod(1 + np.random.randn(n) * 0.01 + 0.003))
tech_up = classifier.calculate_technical_signal(up_prices)

# Downtrending prices
down_prices = pd.Series(100 * np.cumprod(1 + np.random.randn(n) * 0.01 - 0.003))
tech_down = classifier.calculate_technical_signal(down_prices)

# Sideways prices
side_prices = pd.Series(100 + np.random.randn(n) * 2)
tech_side = classifier.calculate_technical_signal(side_prices)

print(f"\nUptrending prices:")
print(f"  Trend: {tech_up['trend']:+.3f}")
print(f"  Momentum: {tech_up['momentum']:+.3f}")
print(f"  Combined: {tech_up['combined']:+.3f}")

print(f"\nDowntrending prices:")
print(f"  Trend: {tech_down['trend']:+.3f}")
print(f"  Momentum: {tech_down['momentum']:+.3f}")
print(f"  Combined: {tech_down['combined']:+.3f}")

print(f"\nSideways prices:")
print(f"  Trend: {tech_side['trend']:+.3f}")
print(f"  Momentum: {tech_side['momentum']:+.3f}")
print(f"  Combined: {tech_side['combined']:+.3f}")

## 5. Combined Alpha Signal Generation

Testing the AlphaCombiner for integrated signal generation.

In [ ]:
from models.predictors.alpha_combiner import AlphaCombiner, CombinedSignal

# Initialize combiner
combiner = AlphaCombiner()

print("Alpha Combiner Configuration:")
print(f"  Weights: {combiner.weights}")
print(f"  Min confidence thresholds: {combiner.MIN_CONFIDENCE}")

In [ ]:
# Add signals from different sources
print("\nAdding Alpha Signals:")
print("="*60)

# On-chain signal (from combined metrics)
onchain = combiner.add_onchain_signal(
    signal=0.4,
    confidence=0.7,
    metadata={'mvrv': 1.5, 'sopr': 1.02, 'netflow': -5000}
)
print(f"  On-chain: signal={onchain.signal:+.2f}, conf={onchain.confidence:.2f}")

# Sentiment signal (from CARVS)
sentiment = combiner.add_sentiment_signal(
    signal=0.3,
    confidence=0.6,
    metadata={'carvs_score': 0.35, 'num_posts': 25}
)
print(f"  Sentiment: signal={sentiment.signal:+.2f}, conf={sentiment.confidence:.2f}")

# Technical signal
technical = combiner.add_technical_signal(
    signal=0.2,
    confidence=0.8,
    metadata={'trend': 0.3, 'momentum': 0.1}
)
print(f"  Technical: signal={technical.signal:+.2f}, conf={technical.confidence:.2f}")

In [ ]:
# Combine signals
result = combiner.combine()

print("\nCombined Signal Result:")
print("="*60)
print(f"  Combined Signal: {result.signal:+.4f}")
print(f"  Confidence: {result.confidence:.4f}")
print(f"  Regime: {result.regime.value}")
print(f"  Regime Confidence: {result.regime_confidence:.4f}")
print(f"  Position Recommendation: {result.position_recommendation}")
print(f"  Position Size Multiplier: {result.position_size_mult:.4f}")

print("\nComponent Signals:")
for name, alpha in result.component_signals.items():
    print(f"  {name:12}: signal={alpha.signal:+.2f}, conf={alpha.confidence:.2f}")

In [ ]:
# Generate trading decision
decision = combiner.generate_trading_decision(current_position='flat')

print("\nTrading Decision:")
print("="*60)
print(f"  Action: {decision['action']}")
print(f"  Reason: {decision['reason']}")
print(f"  Target Position: {decision['target_position']}")
print(f"  Size: {decision['size']:.4f}")
print(f"  Signal: {decision['signal']:+.4f}")
print(f"  Confidence: {decision['confidence']:.4f}")
print(f"  Regime: {decision['regime']}")

In [ ]:
# Test multiple scenarios
print("\nSignal Scenarios:")
print("="*80)

scenarios = [
    {'name': 'Strong Bullish', 'onchain': 0.7, 'sentiment': 0.6, 'technical': 0.5},
    {'name': 'Mildly Bullish', 'onchain': 0.3, 'sentiment': 0.2, 'technical': 0.3},
    {'name': 'Mixed/Neutral', 'onchain': 0.1, 'sentiment': -0.1, 'technical': 0.0},
    {'name': 'Mildly Bearish', 'onchain': -0.3, 'sentiment': -0.2, 'technical': -0.3},
    {'name': 'Strong Bearish', 'onchain': -0.7, 'sentiment': -0.6, 'technical': -0.5},
]

print(f"{'Scenario':<15} {'Signal':>8} {'Conf':>8} {'Regime':<15} {'Recommendation':<12}")
print("-"*80)

for s in scenarios:
    combiner = AlphaCombiner()
    combiner.add_onchain_signal(s['onchain'], confidence=0.7)
    combiner.add_sentiment_signal(s['sentiment'], confidence=0.6)
    combiner.add_technical_signal(s['technical'], confidence=0.8)
    result = combiner.combine()
    
    print(f"{s['name']:<15} {result.signal:+.4f}   {result.confidence:.4f}   "
          f"{result.regime.value:<15} {result.position_recommendation:<12}")

## 6. Integration with Real Data

Testing the full pipeline with real price data.

In [ ]:
from data.storage.timeseries_db import TimeSeriesDB

# Load real price data
db = TimeSeriesDB()
ohlcv = db.fetch_ohlcv('BTC/USD', days=30)
db.close()

print(f"Loaded {len(ohlcv)} price records")
print(f"Date range: {ohlcv['time'].min()} to {ohlcv['time'].max()}")

# Get recent prices
prices = ohlcv['close']
print(f"\nCurrent price: ${prices.iloc[-1]:,.2f}")
print(f"24h change: {(prices.iloc[-1]/prices.iloc[-24] - 1)*100:+.2f}%")

In [ ]:
# Create combiner with technical signal from real data
combiner = AlphaCombiner()

# Calculate technical signal from prices
tech_signal = combiner.calculate_technical_from_prices(prices)

print("Technical Signal from Real Data:")
print(f"  Signal: {tech_signal.signal:+.4f}")
print(f"  Confidence: {tech_signal.confidence:.4f}")
print(f"  Metadata: {tech_signal.metadata}")

In [ ]:
# Add sentiment if we have posts
if multi_posts and 'signal' in dir():
    combiner.add_sentiment_signal(
        signal=signal['signal'],
        confidence=signal['confidence'],
        metadata={'carvs_score': signal.get('carvs_score', 0)}
    )
    print(f"Added sentiment signal: {signal['signal']:+.4f}")
else:
    # Use neutral sentiment if not available
    combiner.add_sentiment_signal(0.0, confidence=0.3)
    print("Using neutral sentiment (no Reddit data)")

# Add placeholder on-chain signal (would come from Dune in production)
combiner.add_onchain_signal(0.0, confidence=0.3, metadata={'source': 'placeholder'})
print("Using placeholder on-chain signal (Dune not configured)")

# Generate final combined signal
final_result = combiner.combine()

print("\n" + "="*60)
print("CURRENT MARKET SIGNAL")
print("="*60)
print(f"  Combined Signal: {final_result.signal:+.4f}")
print(f"  Confidence: {final_result.confidence:.4f}")
print(f"  Market Regime: {final_result.regime.value}")
print(f"  Recommendation: {final_result.position_recommendation}")
print(f"  Size Multiplier: {final_result.position_size_mult:.2f}")

## 7. Summary and Validation

In [ ]:
# Final validation
print("="*60)
print("PHASE 3 VALIDATION RESULTS")
print("="*60)

# Criteria
criteria = {
    'Reddit fetching works': (len(multi_posts) > 0, f"{len(multi_posts)} posts"),
    'Sentiment signals in [-1,1]': (
        -1 <= final_result.component_signals['sentiment'].signal <= 1,
        f"{final_result.component_signals['sentiment'].signal:+.2f}"
    ),
    'Technical signals in [-1,1]': (
        -1 <= final_result.component_signals['technical'].signal <= 1,
        f"{final_result.component_signals['technical'].signal:+.2f}"
    ),
    'Combined signal in [-1,1]': (
        -1 <= final_result.signal <= 1,
        f"{final_result.signal:+.2f}"
    ),
    'Valid regime classification': (
        final_result.regime in MarketRegime,
        final_result.regime.value
    ),
    'Valid position recommendation': (
        final_result.position_recommendation in ['long', 'short', 'flat'],
        final_result.position_recommendation
    ),
}

all_passed = True
for criterion, (passed, value) in criteria.items():
    status = " PASS" if passed else " FAIL"
    print(f"  {criterion}: {status} ({value})")
    if not passed:
        all_passed = False

print("\n" + "="*60)
if all_passed:
    print("ALL VALIDATION CRITERIA PASSED")
else:
    print("SOME CRITERIA FAILED - Review results above")
print("="*60)

In [ ]:
# Summary statistics
print("\n" + "="*60)
print("PHASE 3 ALPHA SOURCES: COMPLETE")
print("="*60)

print(f"\n Reddit Data Source:")
print(f"  - Using public JSON endpoints (no API key required)")
print(f"  - Rate limited to ~10 requests/minute")
print(f"  - Successfully fetched {len(multi_posts)} posts")

print(f"\n Sentiment Analysis:")
print(f"  - FinBERT model loaded successfully")
print(f"  - CARVS scoring implemented")
print(f"  - Generates signals in valid range")

print(f"\n On-Chain Signals:")
print(f"  - MVRV, SOPR, Netflow interpretation ready")
print(f"  - Dune integration ready (configure query IDs)")

print(f"\n Regime Classification:")
print(f"  - 7 market regimes supported")
print(f"  - Trading bias per regime configured")

print(f"\n Alpha Combiner:")
print(f"  - Weights: {combiner.weights}")
print(f"  - Produces actionable trading signals")

print(f"\nReady for Phase 4 (Advanced Models)!")